In [1]:
import os
import pandas as pd
import numpy as np
import torch
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification, AdamW

cuda


/home/acg/amp_project/amp_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define a class for the AMP data that will correctly format the sequence information
# for fine-tuning with huggingface API

class amp_data():
    def __init__(self, df, tokenizer_name='Rostlab/prot_bert_bfd', max_len=200):
        
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)
        self.max_len = max_len
        
        self.seqs, self.labels = self.get_seqs_labels()
        
    def get_seqs_labels(self):        
        # isolate the amino acid sequences and their respective AMP labels
        seqs = list(df['aa_seq'])
        labels = list(df['AMP'].astype(int))
        
#         assert len(seqs) == len(labels)
        return seqs, labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        seq = " ".join("".join(self.seqs[idx].split()))
        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_len)
        
        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])

        return sample

In [3]:
# read in the train dataset
# create an amp_data class of the dataset

df = pd.read_csv('../data/amp_bert/all_veltri.csv', index_col = 0)
df = df.sample(frac=1, random_state = 0)
print(df.head(7))

train_dataset = amp_data(df)

                                                            aa_seq  aa_len  \
AP02151          YEALVTSILGKLTGLWHNDSVDFMGHICYFRRRPKIRRFKLYHEGK...      95   
AP01951                                          FLPLVLGALSGILPKIL      17   
AP00972                                        FLSLIPHAINAVGVHAKHF      19   
AP01261                                           IIEKLVNTALGLLSGL      16   
AP01298                                       GLFTLIKCAYQLIAPTVACN      20   
AP01802                                     RPWAGNGSVHRYTVLSPRLKTQ      22   
UniRef50_Q9UTR1                                SKENSYVEKLLYKQRFYAS      19   

                   AMP  
AP02151           True  
AP01951           True  
AP00972           True  
AP01261           True  
AP01298           True  
AP01802           True  
UniRef50_Q9UTR1  False  


In [4]:
# define the necessary metrics for performance evaluation

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
#     conf = confusion_matrix(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
#         'confusion matrix': conf
    }

In [5]:
# define the initializing function for Trainer in huggingface

def model_init():
    return AutoModelForSequenceClassification.from_pretrained('Rostlab/prot_bert_bfd')

In [6]:
# training on entire data
# no evaluation/validation

training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=15,              
    learning_rate=5e-5,              
    per_device_train_batch_size=1,   
    warmup_steps=0,               
    weight_decay=0.1,               
    logging_dir='./logs',            
    logging_steps=100,               
    do_train=True,                   
    do_eval=True,                   
    evaluation_strategy="no",    
    save_strategy='no',
    gradient_accumulation_steps=64,  
    fp16=True,                       
    fp16_opt_level="O2",             
    run_name="AMP-BERT",             
    seed=0,                          
    load_best_model_at_end = True
)

trainer = Trainer(
    model_init=model_init,                
    args=training_args,                   
    train_dataset=train_dataset,          
    compute_metrics = compute_metrics,    
)

trainer.train()
# trainer.save_model('/home/hansol/amp/model/')

/home/acg/amp_project/amp_project/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert_bfd and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert_bfd and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/acg/amp_project/amp_project/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but a

Step,Training Loss


KeyboardInterrupt: 